In [1]:
# ! pip install pyspark

In [2]:
import pyspark

from pyspark.sql import SparkSession, Row, functions

In [3]:
sc = pyspark.SparkContext(appName='Union')

23/03/03 14:08:53 WARN Utils: Your hostname, celeron resolves to a loopback address: 127.0.1.1; using 192.168.10.58 instead (on interface wlo1)
23/03/03 14:08:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 14:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
dir(sc)

['PACKAGE_EXTENSIONS',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_assert_on_driver',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_encryption_enabled',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_repr_html_',
 '_serialize_to_jvm',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addArchive',
 'addFile',
 'addPyFile',
 'appName',
 'app

In [5]:
se = SparkSession(sc)

https://medium.com/analytics-vidhya/how-does-pyspark-work-step-by-step-with-pictures-c011402ccd57

# Read custom data

In [6]:
text = se.read.text("data/words.txt")

# rdd = sc.parallelize(text.collect())

In [7]:
text.count() # return number of rows

1

In [9]:
text.take(1) # look at first n rows

In [10]:
text.toPandas() # convert to Pandas

,value
0,"peaceful,*,*,capitalistically, ,necessarianism..."


**Let's found out the size of our text file**

Map applies a function to each element

In [11]:
text_len = text.rdd.map(lambda x: len(x.value))

In [12]:
text_len.collect()

[7667925]

We need to split the file to rows to process its content

flatMap() combines mapping and flattening. It runs the map(), and then the flatten() method to generate the result. We create a single collection

In [18]:
# text.rdd.map(lambda x: x.value.split(',')).collect()

In [13]:
words = text.rdd.flatMap(lambda x: x.value.split(','))

In [14]:
words.collect()

['peaceful',
 '*',
 '*',
 'capitalistically',
 ' ',
 'necessarianism',
 'turbinella',
 'suingly',
 'coexecutrix',
 'nitroxyl',
 'introversive',
 'throttling',
 'feathering',
 'provisory',
 'ileostomy',
 '*',
 'orthometric',
 ' ',
 'shechita',
 ' ',
 'adorableness',
 'gitana',
 ' ',
 'suingly',
 '*',
 'optometry',
 '*',
 'icterine',
 'obdurately',
 'waterfronts',
 'spiriting',
 'kettledrums',
 'fissiparous',
 'terroriser',
 'neutralizer',
 'peaceful',
 'knowable',
 'investigable',
 'spiriting',
 'turbinella',
 'semihardy',
 'piggiest',
 'neutralizer',
 'feathering',
 'piggiest',
 'toastmistress',
 'obligately',
 'orthometric',
 'collaborators',
 'peace',
 ' ',
 'piggiest',
 'peaceful',
 'kielbasas',
 'kettledrums',
 'orthometric',
 'obdurately',
 'obligately',
 ' ',
 'kinetoplast',
 'obligately',
 'shechita',
 'shechita',
 'geophilidae',
 'farrieries',
 '*',
 'toastmistress',
 'knowable',
 'punchinello',
 'ileostomy',
 'turbinella',
 'feathering',
 ' ',
 'terroriser',
 ' ',
 'toastmistr

# Operations

## Count unique words

In [19]:
from operator import add, itemgetter


unique_words_cnt = words.map(lambda word: (word, 1))

In [23]:
unique_words_cnt.reduceByKey(add).sortBy(itemgetter(1)).collect()

[('!', 1),
 ('rinderpest', 1191),
 ('reconnect', 1239),
 ('rhinencephala', 1258),
 ('rabiator', 1267),
 ('reactionaries', 1295),
 ('adorableness', 2438),
 ('admiration', 2498),
 ('anthropographic', 2513),
 ('attractable', 2525),
 ('acantholysis', 2526),
 ('wittier', 3700),
 ('waterfronts', 3738),
 ('wranglership', 3751),
 ('wanderer', 3765),
 ('workingwomen', 3796),
 ('gormandizers', 6204),
 ('gonadotrophic', 6226),
 ('geophilidae', 6251),
 ('gnarliest', 6257),
 ('gitana', 6312),
 ('nitroxyl', 7383),
 ('nonresonant', 7405),
 ('neutralizer', 7540),
 ('necessarianism', 7584),
 ('neencephalon', 7588),
 ('icterine', 8669),
 ('idola', 8715),
 ('introversive', 8763),
 ('ileostomy', 8773),
 ('investigable', 8830),
 ('knowable', 9919),
 ('knightliness', 9979),
 ('kielbasas', 10017),
 ('kinetoplast', 10030),
 ('kettledrums', 10055),
 ('completers', 11152),
 ('cocoanut', 11188),
 ('coexecutrix', 11286),
 ('capitalistically', 11292),
 ('collaborators', 11332),
 ('farrieries', 13651),
 ('finitude'

rewrite

In [24]:
unique_wc = (
    text.rdd
    .flatMap(lambda x: x.value.split(','))
    .map(lambda word: (word, 1))
    .reduceByKey(add)
)

In [26]:
unique_wc.collect()

[('*', 62500),
 ('turbinella', 18812),
 ('coexecutrix', 11286),
 ('introversive', 8763),
 ('throttling', 18821),
 ('feathering', 13745),
 ('orthometric', 17426),
 ('gitana', 6312),
 ('optometry', 17411),
 ('icterine', 8669),
 ('obdurately', 17598),
 ('waterfronts', 3738),
 ('kettledrums', 10055),
 ('fissiparous', 13744),
 ('neutralizer', 7540),
 ('knowable', 9919),
 ('investigable', 8830),
 ('semihardy', 19937),
 ('piggiest', 15983),
 ('toastmistress', 18708),
 ('peace', 16209),
 ('geophilidae', 6251),
 ('farrieries', 13651),
 ('finitude', 13699),
 ('wittier', 3700),
 ('\t', 25000),
 ('anthropographic', 2513),
 ('synonyme', 20054),
 ('attractable', 2525),
 ('idola', 8715),
 ('admiration', 2498),
 ('rhinencephala', 1258),
 ('reconnect', 1239),
 ('reactionaries', 1295),
 ('!', 1),
 ('peaceful', 16379),
 ('capitalistically', 11292),
 (' ', 75000),
 ('necessarianism', 7584),
 ('suingly', 19999),
 ('nitroxyl', 7383),
 ('provisory', 16365),
 ('ileostomy', 8773),
 ('shechita', 20058),
 ('ador

You can watch the plan of calculations

In [27]:
print(unique_wc.toDebugString().decode('utf-8'))

(2) PythonRDD[54] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[53] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[52] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[51] at reduceByKey at /tmp/ipykernel_8913/870653427.py:2 []
    |  PythonRDD[50] at reduceByKey at /tmp/ipykernel_8913/870653427.py:2 []
    |  MapPartitionsRDD[16] at javaToPython at NativeMethodAccessorImpl.java:0 []
    |  MapPartitionsRDD[15] at javaToPython at NativeMethodAccessorImpl.java:0 []
    |  SQLExecutionRDD[14] at javaToPython at NativeMethodAccessorImpl.java:0 []
    |  MapPartitionsRDD[13] at javaToPython at NativeMethodAccessorImpl.java:0 []
    |  FileScanRDD[10] at toPandas at /tmp/ipykernel_8913/1428240239.py:1 []


## Count unique char bi-grams

In [28]:
letters = 'abcdegf'

In [30]:
[letters[i:i+2] for i in range(len(letters) - len(letters) % 2)]

['ab', 'bc', 'cd', 'de', 'eg', 'gf']

In [31]:
bigrams = words.flatMap(lambda letters: [letters[i:i+2] for i in range(len(letters) - len(letters) % 2)]).map(lambda bigramm: (bigramm, 1)).reduceByKey(add)

In [33]:
bigrams.sortBy(lambda x: x[-1]).collect()

[('rp', 1191),
 ('nn', 1239),
 ('rh', 1258),
 ('dm', 2498),
 ('gr', 2513),
 ('po', 2513),
 ('og', 2513),
 ('ys', 2526),
 ('wi', 3700),
 ('fr', 3738),
 ('rf', 3738),
 ('ts', 3738),
 ('p', 3751),
 ('wr', 3751),
 ('io', 3793),
 ('rk', 3796),
 ('gw', 3796),
 ('rm', 6204),
 ('ma', 6204),
 ('di', 6204),
 ('da', 6251),
 ('ge', 6251),
 ('ae', 6251),
 ('gn', 6257),
 ('rl', 6257),
 ('ox', 7383),
 ('xy', 7383),
 ('yl', 7383),
 ('nr', 7405),
 ('eu', 7540),
 ('sm', 7584),
 ('m', 7584),
 ('ee', 7588),
 ('wo', 7592),
 ('iv', 8763),
 ('my', 8773),
 ('nv', 8830),
 ('ep', 8846),
 ('ow', 9919),
 ('gh', 9979),
 ('ht', 9979),
 ('lb', 10017),
 ('ba', 10017),
 ('um', 10055),
 ('ke', 10055),
 ('dr', 10055),
 ('ru', 10055),
 ('ms', 10055),
 ('mp', 11152),
 ('nd', 11160),
 ('ad', 11162),
 ('oc', 11188),
 ('nu', 11188),
 ('oe', 11286),
 ('ex', 11286),
 ('cu', 11286),
 ('xe', 11286),
 ('ix', 11286),
 ('bo', 11332),
 ('go', 12430),
 ('fa', 13651),
 ('ud', 13699),
 ('ct', 13728),
 ('pa', 13744),
 ('fe', 13745),
 ('

## How many unique words in total

If we want to calculate the count of unique words. 

1. Use *aggregate* function

In [34]:
words = text.rdd.flatMap(lambda x: x.value.split(','))

In [ ]:
words.aggregate()

In [37]:
unique_words_num = (
    words
    .map(lambda x: (x, 1))
    .reduceByKey(max)
    .aggregate(0, lambda x, y: x + y[1], lambda x, y: x + y)
)

In [38]:
unique_words_num

69

or *count()* function

In [39]:
unique_words_num = (
    words
    .map(lambda x: (x, 1))
    .reduceByKey(add)
    .count()
)

In [40]:
unique_words_num

69

**Use converting to DataFrame**

In [44]:
grouped_words = (
    words
    .map(lambda x: (x, 1))
    .toDF(['word', 'cnt'])
    .groupBy('word')
)

In [47]:
grouped_words.agg(functions.count('word')).collect()

[Row(word='filiated', count(word)=13911),
 Row(word='knightliness', count(word)=9979),
 Row(word='shechita', count(word)=20058),
 Row(word='acantholysis', count(word)=2526),
 Row(word='nitroxyl', count(word)=7383),
 Row(word='necessarianism', count(word)=7584),
 Row(word='kinetoplast', count(word)=10030),
 Row(word='*', count(word)=62500),
 Row(word='wranglership', count(word)=3751),
 Row(word='!', count(word)=1),
 Row(word='gonadotrophic', count(word)=6226),
 Row(word='gormandizers', count(word)=6204),
 Row(word='gitana', count(word)=6312),
 Row(word='\t', count(word)=25000),
 Row(word='peaceful', count(word)=16379),
 Row(word='feathering', count(word)=13745),
 Row(word='synonyme', count(word)=20054),
 Row(word='coexecutrix', count(word)=11286),
 Row(word='adorableness', count(word)=2438),
 Row(word='wittier', count(word)=3700),
 Row(word='turbinella', count(word)=18812),
 Row(word='cocoanut', count(word)=11188),
 Row(word='icterine', count(word)=8669),
 Row(word='fissiparous', count(

In [48]:
unique_words_cnt = (
    words
    .map(lambda x: (1, x))
    .toDF(['_', 'word'])
    .groupby('_')
    .agg(functions.countDistinct('word'))
)

In [49]:
unique_words_cnt.collect()

[Row(_=1, count(word)=69)]

In [50]:
unique_words_cnt.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(3) HashAggregate(keys=[_#31L], functions=[count(distinct word#32)])
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 1
         +- Exchange hashpartitioning(_#31L, 200), ENSURE_REQUIREMENTS, [plan_id=143]
            +- *(2) HashAggregate(keys=[_#31L], functions=[partial_count(distinct word#32)])
               +- *(2) HashAggregate(keys=[_#31L, word#32], functions=[])
                  +- AQEShuffleRead coalesced
                     +- ShuffleQueryStage 0
                        +- Exchange hashpartitioning(_#31L, word#32, 200), ENSURE_REQUIREMENTS, [plan_id=119]
                           +- *(1) HashAggregate(keys=[_#31L, word#32], functions=[])
                              +- *(1) Scan ExistingRDD[_#31L,word#32]
+- == Initial Plan ==
   HashAggregate(keys=[_#31L], functions=[count(distinct word#32)])
   +- Exchange hashpartitioning(_#31L, 200), ENSURE_REQUIREMENTS, [plan_id=105]
      +- Hash

**add filter**

Add some condition on data. For example, work only with words which have at least 2 characters

In [51]:
unique_filtered_words_num = (
    words
    .filter(lambda x: len(x) > 1)
    .map(lambda x: (x, 1))
    .reduceByKey(add)
    .count()
)

In [52]:
unique_filtered_words_num

65

## Solve wordCount with different reducers 

In [53]:
df_wc = sc.parallelize(words.map(lambda x: (x, 1)).collect(), 16)  # it won't actually parallelize on local machine with

### reduceByKey()

In [54]:
wc_reduce = (
    df_wc
    .reduceByKey(add)
)

In [55]:
wc_reduce.collect()

[('*', 62500),
 ('knowable', 9919),
 ('finitude', 13699),
 ('attractable', 2525),
 ('admiration', 2498),
 ('shechita', 20058),
 ('tinamous', 18884),
 ('filiated', 13911),
 ('gormandizers', 6204),
 ('workingwomen', 3796),
 ('obdurately', 17598),
 ('fissiparous', 13744),
 ('farrieries', 13651),
 ('!', 1),
 ('ileostomy', 8773),
 ('spiriting', 19952),
 ('completers', 11152),
 ('ostracize', 17516),
 ('wranglership', 3751),
 ('orthometric', 17426),
 ('optometry', 17411),
 ('semihardy', 19937),
 ('anthropographic', 2513),
 ('capitalistically', 11292),
 ('suingly', 19999),
 ('adorableness', 2438),
 ('kielbasas', 10017),
 ('neencephalon', 7588),
 ('rabiator', 1267),
 ('throttling', 18821),
 ('investigable', 8830),
 ('peace', 16209),
 ('rhinencephala', 1258),
 ('peaceful', 16379),
 (' ', 75000),
 ('gonadotrophic', 6226),
 ('rinderpest', 1191),
 ('coexecutrix', 11286),
 ('introversive', 8763),
 ('feathering', 13745),
 ('kettledrums', 10055),
 ('neutralizer', 7540),
 ('geophilidae', 6251),
 ('idol

### groupByKey()

In [56]:
wc_groupby = (
    df_wc
    .groupByKey()
    .map(lambda x: (x[0], sum(x[1])))
)

In [57]:
wc_groupby.collect()

[('*', 62500),
 ('knowable', 9919),
 ('finitude', 13699),
 ('attractable', 2525),
 ('admiration', 2498),
 ('shechita', 20058),
 ('tinamous', 18884),
 ('filiated', 13911),
 ('gormandizers', 6204),
 ('workingwomen', 3796),
 ('obdurately', 17598),
 ('fissiparous', 13744),
 ('farrieries', 13651),
 ('!', 1),
 ('ileostomy', 8773),
 ('spiriting', 19952),
 ('completers', 11152),
 ('ostracize', 17516),
 ('wranglership', 3751),
 ('orthometric', 17426),
 ('optometry', 17411),
 ('semihardy', 19937),
 ('anthropographic', 2513),
 ('capitalistically', 11292),
 ('suingly', 19999),
 ('adorableness', 2438),
 ('kielbasas', 10017),
 ('neencephalon', 7588),
 ('rabiator', 1267),
 ('throttling', 18821),
 ('investigable', 8830),
 ('peace', 16209),
 ('rhinencephala', 1258),
 ('peaceful', 16379),
 (' ', 75000),
 ('gonadotrophic', 6226),
 ('rinderpest', 1191),
 ('coexecutrix', 11286),
 ('introversive', 8763),
 ('feathering', 13745),
 ('kettledrums', 10055),
 ('neutralizer', 7540),
 ('geophilidae', 6251),
 ('idol

**GroupByKey() vs ReduceByKey()**

![](https://sparkbyexamples.com/wp-content/uploads/2022/12/groupByKey-Vs-reduceByKey-1-1.jpg)

https://sparkbyexamples.com/spark/spark-groupbykey/

From the above diagram, when calling Spark `groupByKey` on the dataset,

- All the key-value pairs are shuffled across the executors.
- This is a lot of unnecessary data to be transferred over the network.
- Spark uses the partition function on the dataset to determine which partition to be shuffled across the executors.
- Spark `groupByKey` spills data to disk when there is more data shuffled onto a single executor machine than can fit in memory.
- If the size of the data is more than the memory of the executor Spark spills out the data to disk one key at a time – so if a single key has more key-value pairs than can fit in executor memory, an out-of-memory exception occurs.
- This was handled in a later release of Spark so the job can still proceed, but should still be avoided – when Spark needs to spill to disk, performance is severely impacted.

Whereas when we use `reduceByKey` on the same datasets,

- key-value pairs on the same machine with the same key are combined (by using the lambda function passed into `reduceByKey` ) before the data is shuffled.
- The lambda function is called again to reduce all the values from each partition to produce one final result.
- In this way, shuffling is minimal in the case of `ReduceByKey` and improvises performance a lot faster.

### combineByKey()

In [39]:
wc_combine = (
    df_wc
    .combineByKey(
        lambda count: count,  # create combiner
        lambda counts_agg, count: counts_agg + count,  # merge values in each partitions
        lambda counts_agg_one, counts_agg_two: counts_agg_one + counts_agg_two  # combine values from different partitions
    )  
)

In [40]:
wc_combine.collect()

[('*', 62500),
 ('knowable', 9919),
 ('finitude', 13699),
 ('attractable', 2525),
 ('admiration', 2498),
 ('shechita', 20058),
 ('tinamous', 18884),
 ('filiated', 13911),
 ('gormandizers', 6204),
 ('workingwomen', 3796),
 ('obdurately', 17598),
 ('fissiparous', 13744),
 ('farrieries', 13651),
 ('!', 1),
 ('ileostomy', 8773),
 ('spiriting', 19952),
 ('completers', 11152),
 ('ostracize', 17516),
 ('wranglership', 3751),
 ('orthometric', 17426),
 ('optometry', 17411),
 ('semihardy', 19937),
 ('anthropographic', 2513),
 ('capitalistically', 11292),
 ('suingly', 19999),
 ('adorableness', 2438),
 ('kielbasas', 10017),
 ('neencephalon', 7588),
 ('rabiator', 1267),
 ('throttling', 18821),
 ('investigable', 8830),
 ('peace', 16209),
 ('rhinencephala', 1258),
 ('peaceful', 16379),
 (' ', 75000),
 ('gonadotrophic', 6226),
 ('rinderpest', 1191),
 ('coexecutrix', 11286),
 ('introversive', 8763),
 ('feathering', 13745),
 ('kettledrums', 10055),
 ('neutralizer', 7540),
 ('geophilidae', 6251),
 ('idol

https://www.oreilly.com/library/view/data-algorithms-with/9781492082378/ch04.html#callout_reductions_in_spark_CO6-1

## Calculate most popular lengths of words

And return the sorted answer in descending order 

In [58]:
most_popular_lens = (
    words
    .map(lambda x: (len(x), 1))
    .reduceByKey(add)
    .sortBy(lambda x: -x[1])
)

In [59]:
most_popular_lens.collect()

[(1, 162501),
 (8, 161159),
 (10, 133542),
 (9, 117467),
 (11, 106314),
 (12, 53875),
 (13, 38819),
 (5, 24924),
 (7, 23699),
 (16, 11292),
 (14, 7584),
 (6, 6312),
 (15, 2513)]

**Now return the same but with word examples in additional field**

In [60]:
most_popular_lens = (
    words
    .map(lambda x: (len(x), (1, {x})))
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1].union(y[1])))
    .map(lambda x: (x[0], x[1][0], str(x[1][1])))
    .sortBy(lambda x: -x[1])
)

In [61]:
most_popular_lens.collect()

[(1, 162501, "{'*', '!', '\\t', ' '}"),
 (8,
  161159,
  "{'knowable', 'shechita', 'piggiest', 'nitroxyl', 'peaceful', 'wanderer', 'synonyme', 'finitude', 'tinamous', 'filiated', 'rabiator', 'icterine', 'cocoanut'}"),
 (10,
  133542,
  "{'admiration', 'farrieries', 'completers', 'throttling', 'feathering', 'obligately', 'turbinella', 'terroriser', 'obdurately', 'rinderpest'}"),
 (9,
  117467,
  "{'ileostomy', 'optometry', 'ostracize', 'semihardy', 'reconnect', 'gnarliest', 'spiriting', 'kielbasas', 'provisory'}"),
 (11,
  106314,
  "{'fissiparous', 'kettledrums', 'geophilidae', 'coexecutrix', 'neutralizer', 'kinetoplast', 'attractable', 'orthometric', 'nonresonant', 'waterfronts', 'punchinello'}"),
 (12,
  53875,
  "{'gormandizers', 'workingwomen', 'adorableness', 'investigable', 'introversive', 'knightliness', 'wranglership', 'neencephalon', 'acantholysis'}"),
 (13,
  38819,
  "{'reactionaries', 'toastmistress', 'rhinencephala', 'gonadotrophic', 'collaborators'}"),
 (5, 24924, "{'idol

In [63]:
most_popular_lens.toDF(['len', 'words_count', 'words_examples']).toPandas().set_index('len')

,words_count,words_examples
len,,
1,162501,"{'*', '!', '\t', ' '}"
8,161159,"{'knowable', 'shechita', 'piggiest', 'nitroxyl..."
10,133542,"{'admiration', 'farrieries', 'completers', 'th..."
9,117467,"{'ileostomy', 'optometry', 'ostracize', 'semih..."
11,106314,"{'fissiparous', 'kettledrums', 'geophilidae', ..."
12,53875,"{'gormandizers', 'workingwomen', 'adorableness..."
13,38819,"{'reactionaries', 'toastmistress', 'rhinenceph..."
5,24924,"{'idola', 'peace'}"
7,23699,"{'wittier', 'suingly'}"


## How words are usually ending? 

Count the frequency of last symbols. Sort result in ascending order and cast to dataframe

In [64]:
end_chars = (
    words
    .map(lambda x: (x[-1], 1))
    .reduceByKey(add)
    .sortBy(lambda x: x[1])
)

In [65]:
end_chars.collect()

[('!', 1),
 ('p', 3751),
 ('m', 7584),
 ('x', 11286),
 ('n', 13882),
 ('d', 13911),
 ('o', 16314),
 ('l', 23762),
 ('\t', 25000),
 ('c', 26165),
 ('r', 34797),
 ('g', 52518),
 ('t', 53293),
 ('a', 55155),
 ('*', 62500),
 (' ', 75000),
 ('e', 112435),
 ('y', 128924),
 ('s', 133723)]

In [67]:
end_chars.toDF(['char', 'cnt']).take(5)

[Row(char='!', cnt=1),
 Row(char='p', cnt=3751),
 Row(char='m', cnt=7584),
 Row(char='x', cnt=11286),
 Row(char='n', cnt=13882)]

With Pandas

In [69]:
end_chars.toDF(['char', 'cnt']).toPandas().set_index('char')

,cnt
char,
!,1
p,3751
m,7584
x,11286
n,13882
d,13911
o,16314
l,23762
\t,25000


# Now let's repeat some of it with SQL

Registering a table

In [71]:
df_words = words.map(lambda x: (x, 1)).toDF(['word', 'cnt'])

df_words.registerTempTable("words")

## How many unique words in total

In [72]:
unique_words_cnt = se.sql(f"""
    select count(distinct word)
    from words
"""
)

In [73]:
unique_words_cnt.collect()

[Row(count(DISTINCT word)=69)]

Or using `select` and the function from `pyspark.sql.functions`

In [76]:
df_words.select(functions.countDistinct('word')).show()

+--------------------+
|count(DISTINCT word)|
+--------------------+
|                  69|
+--------------------+



**Add filter on word len**

In [77]:
unique_words_cnt = se.sql(f"""
    select count(distinct word)
    from words
    where length(word) > 1
"""
)

In [79]:
unique_words_cnt.show()

+--------------------+
|count(DISTINCT word)|
+--------------------+
|                  65|
+--------------------+



Do the same in functional style. Search for `where` and `select` methods

In [80]:
df_words.where('length(word) > 1').select(functions.countDistinct('word')).show()

+--------------------+
|count(DISTINCT word)|
+--------------------+
|                  65|
+--------------------+



## Count number of word lengths

And order in ascending order of counts

In [81]:
wc = se.sql(f"""
    select length(word) as word_len, count(1) as cnt
    from words
    group by 1
    order by 2 asc
"""
)

In [85]:
wc.collect()

[Row(word_len=15, cnt=2513),
 Row(word_len=6, cnt=6312),
 Row(word_len=14, cnt=7584),
 Row(word_len=16, cnt=11292),
 Row(word_len=7, cnt=23699),
 Row(word_len=5, cnt=24924),
 Row(word_len=13, cnt=38819),
 Row(word_len=12, cnt=53875),
 Row(word_len=11, cnt=106314),
 Row(word_len=9, cnt=117467),
 Row(word_len=10, cnt=133542),
 Row(word_len=8, cnt=161159),
 Row(word_len=1, cnt=162501)]

In [86]:
wc.show()

+--------+------+
|word_len|   cnt|
+--------+------+
|      15|  2513|
|       6|  6312|
|      14|  7584|
|      16| 11292|
|       7| 23699|
|       5| 24924|
|      13| 38819|
|      12| 53875|
|      11|106314|
|       9|117467|
|      10|133542|
|       8|161159|
|       1|162501|
+--------+------+



## Count unique words

Count the number of occurrences for each word, sort in descending order and filter words with total count < 10000

In [87]:
wc = se.sql(f"""
    select word, count(1) as cnt
    from words
    group by 1
    having cnt > 10000
    order by 2 desc
"""
)

In [88]:
wc.collect()

[Row(word=' ', cnt=75000),
 Row(word='*', cnt=62500),
 Row(word='\t', cnt=25000),
 Row(word='shechita', cnt=20058),
 Row(word='synonyme', cnt=20054),
 Row(word='suingly', cnt=19999),
 Row(word='spiriting', cnt=19952),
 Row(word='semihardy', cnt=19937),
 Row(word='tinamous', cnt=18884),
 Row(word='throttling', cnt=18821),
 Row(word='turbinella', cnt=18812),
 Row(word='toastmistress', cnt=18708),
 Row(word='terroriser', cnt=18525),
 Row(word='obdurately', cnt=17598),
 Row(word='obligately', cnt=17549),
 Row(word='ostracize', cnt=17516),
 Row(word='orthometric', cnt=17426),
 Row(word='optometry', cnt=17411),
 Row(word='peaceful', cnt=16379),
 Row(word='provisory', cnt=16365),
 Row(word='punchinello', cnt=16314),
 Row(word='peace', cnt=16209),
 Row(word='piggiest', cnt=15983),
 Row(word='filiated', cnt=13911),
 Row(word='feathering', cnt=13745),
 Row(word='fissiparous', cnt=13744),
 Row(word='finitude', cnt=13699),
 Row(word='farrieries', cnt=13651),
 Row(word='collaborators', cnt=11332),


In [89]:
wc.show()

+-------------+-----+
|         word|  cnt|
+-------------+-----+
|             |75000|
|            *|62500|
|           \t|25000|
|     shechita|20058|
|     synonyme|20054|
|      suingly|19999|
|    spiriting|19952|
|    semihardy|19937|
|     tinamous|18884|
|   throttling|18821|
|   turbinella|18812|
|toastmistress|18708|
|   terroriser|18525|
|   obdurately|17598|
|   obligately|17549|
|    ostracize|17516|
|  orthometric|17426|
|    optometry|17411|
|     peaceful|16379|
|    provisory|16365|
+-------------+-----+
only showing top 20 rows



## Join data from other file

Add descriptions from file words2def.txt, convert them to dataframe with (key, value)

In [90]:
defs = se.read.text("data/word2def.txt")

In [91]:
defs.collect()

[Row(value='peace | the nonwarring condition of a nation, group of nations, or the world.'),
 Row(value='peaceful | characterized by peace; free from war, strife, commotion, violence, or disorder.'),
 Row(value='reconnect | to link or be linked together again'),
 Row(value='reactionaries | plural for reactionary (adj. of, pertaining to, marked by, or favoring reaction, especially extreme conservatism or rightism in politics; opposing political or social change.)'),
 Row(value='admiration | a feeling of wonder, pleasure, or approval.'),
 Row(value='kettledrums | drums consisting of a hollow hemisphere of brass, copper, or fiberglass over which is stretched a skin, the tension of which can be modified by hand screws or foot pedals to vary the pitch.'),
 Row(value='cocoanut | coconut.'),
 Row(value='finitude | a finite state or quality.'),
 Row(value='investigable | capable of being investigated.'),
 Row(value='hope | the feeling that what is wanted can be had or that events will turn out

Create a dictionary dataframe and register the corresponding temp table

In [92]:
dictionary = defs.rdd.map(lambda x: x.value.split('|')).map(lambda x: (x[0].strip(), x[1].strip()))

In [93]:
dictionary = dictionary.toDF(['word', 'definition'])
dictionary.createOrReplaceTempView('dictionary')

In [94]:
dictionary.show()

+-------------+--------------------+
|         word|          definition|
+-------------+--------------------+
|        peace|the nonwarring co...|
|     peaceful|characterized by ...|
|    reconnect|to link or be lin...|
|reactionaries|plural for reacti...|
|   admiration|a feeling of wond...|
|  kettledrums|drums consisting ...|
|     cocoanut|            coconut.|
|     finitude|a finite state or...|
| investigable|capable of being ...|
|         hope|the feeling that ...|
+-------------+--------------------+



**Repeat previous task. Leave only words, for which we have description**

Using Spark SQL

In [95]:
wc_filtered = se.sql(f"""
    select word, definition, count(1) as cnt
    from words
    inner join dictionary using(word)
    group by 1, 2
    order by 3 desc
"""
)

In [96]:
wc_filtered.collect()

[Row(word='peaceful', definition='characterized by peace; free from war, strife, commotion, violence, or disorder.', cnt=16379),
 Row(word='peace', definition='the nonwarring condition of a nation, group of nations, or the world.', cnt=16209),
 Row(word='finitude', definition='a finite state or quality.', cnt=13699),
 Row(word='cocoanut', definition='coconut.', cnt=11188),
 Row(word='kettledrums', definition='drums consisting of a hollow hemisphere of brass, copper, or fiberglass over which is stretched a skin, the tension of which can be modified by hand screws or foot pedals to vary the pitch.', cnt=10055),
 Row(word='investigable', definition='capable of being investigated.', cnt=8830),
 Row(word='admiration', definition='a feeling of wonder, pleasure, or approval.', cnt=2498),
 Row(word='reactionaries', definition='plural for reactionary (adj. of, pertaining to, marked by, or favoring reaction, especially extreme conservatism or rightism in politics; opposing political or social ch

**Leave all words. For words where we don't have descriptions, write 'None'**

In [98]:
se.sql(f"""
    select word, coalesce(definition, 'None') as def, count(1) as cnt
    from words
    left join dictionary using(word)
    group by 1, 2
    order by 3 desc
"""
).collect()

[Row(word=' ', def='None', cnt=75000),
 Row(word='*', def='None', cnt=62500),
 Row(word='\t', def='None', cnt=25000),
 Row(word='shechita', def='None', cnt=20058),
 Row(word='synonyme', def='None', cnt=20054),
 Row(word='suingly', def='None', cnt=19999),
 Row(word='spiriting', def='None', cnt=19952),
 Row(word='semihardy', def='None', cnt=19937),
 Row(word='tinamous', def='None', cnt=18884),
 Row(word='throttling', def='None', cnt=18821),
 Row(word='turbinella', def='None', cnt=18812),
 Row(word='toastmistress', def='None', cnt=18708),
 Row(word='terroriser', def='None', cnt=18525),
 Row(word='obdurately', def='None', cnt=17598),
 Row(word='obligately', def='None', cnt=17549),
 Row(word='ostracize', def='None', cnt=17516),
 Row(word='orthometric', def='None', cnt=17426),
 Row(word='optometry', def='None', cnt=17411),
 Row(word='peaceful', def='characterized by peace; free from war, strife, commotion, violence, or disorder.', cnt=16379),
 Row(word='provisory', def='None', cnt=16365),
 R

**Calculate last letters popularity**

In [99]:
se.sql("""
    select right(word, 1) as last_char, count(1)
    from words
    group by 1
    order by 2 desc
"""
).show()

+---------+--------+
|last_char|count(1)|
+---------+--------+
|        s|  133723|
|        y|  128924|
|        e|  112435|
|         |   75000|
|        *|   62500|
|        a|   55155|
|        t|   53293|
|        g|   52518|
|        r|   34797|
|        c|   26165|
|       \t|   25000|
|        l|   23762|
|        o|   16314|
|        d|   13911|
|        n|   13882|
|        x|   11286|
|        m|    7584|
|        p|    3751|
|        !|       1|
+---------+--------+

